# **EDA del Proyecto 1: MLOps**

#### Importamos Librerias 

In [62]:
import pandas as pd
import matplotlib.pyplot as plt

## Buscando nulos y duplicados

#### **Parte Steam_games**

Abrimos el archivo 'steam_games.csv'

In [63]:
df_games = pd.read_csv(".\datasets\steam_games.csv", sep=";", encoding="utf-8")
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28851 non-null  object 
 1   app_name      32133 non-null  object 
 2   url           32133 non-null  object 
 3   release_date  30067 non-null  object 
 4   price         30757 non-null  object 
 5   id            32132 non-null  float64
 6   developer     28836 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.7+ MB


Observamos que la columna 'id' esta en el tipo de dato equivocado (float), ahora lo pasaremos a int  
Pero primero tendremos de encangarnos de los nulos (NaN) que tiene:

In [64]:
df_games.head(3)

,genres,app_name,url,release_date,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,4.99,761140.0,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,Free To Play,643980.0,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,Free to Play,670290.0,Poolians.com


In [65]:
df_games[df_games["id"].isna()]

,genres,app_name,url,release_date,price,id,developer
30959,"['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,19.99,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


En la columna 'url' podemos ver el id del producto, asi que lo meteremos en la columna id

In [66]:
df_games.loc[30959, "id"] = 200260
df_games.loc[30959]

genres                                 ['Action', 'Adventure']
app_name        Batman: Arkham City - Game of the Year Edition
url                   http://store.steampowered.com/app/200260
release_date                                        2012-09-07
price                                                    19.99
id                                                    200260.0
developer           Rocksteady Studios,Feral Interactive (Mac)
Name: 30959, dtype: object

In [67]:
df_games["id_item"] = df_games["id"].astype(int)
df_games.head(3)

,genres,app_name,url,release_date,price,id,developer,id_item
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,4.99,761140.0,Kotoshiro,761140
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,Free To Play,643980.0,Secret Level SRL,643980
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,Free to Play,670290.0,Poolians.com,670290


Ahora eliminaremos todas las columnas que no son necesarias para el modelo

In [68]:
a_eliminar = ["url", "url", "id"]

df_games = df_games.drop(a_eliminar, axis=1)

In [70]:
df_games.head(4)

,genres,app_name,release_date,price,developer,id_item
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,4.99,Kotoshiro,761140
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,Free To Play,Secret Level SRL,643980
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,Free to Play,Poolians.com,670290
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,0.99,彼岸领域,767400


### **Tomaremos acción contra los nulos del dataframe**

In [71]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28851 non-null  object
 1   app_name      32133 non-null  object
 2   release_date  30067 non-null  object
 3   price         30757 non-null  object
 4   developer     28836 non-null  object
 5   id_item       32133 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 1.3+ MB


In [72]:
df_games["genres"] = df_games["genres"].fillna("Sin Dato")

df_games["release_date"] = df_games["release_date"].fillna("Sin Dato")

df_games["price"] = df_games["price"].fillna("Sin Dato")

df_games["developer"] = df_games["developer"].fillna("Sin Dato")

df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        32133 non-null  object
 1   app_name      32133 non-null  object
 2   release_date  32133 non-null  object
 3   price         32133 non-null  object
 4   developer     32133 non-null  object
 5   id_item       32133 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 1.3+ MB


Intentamos ver si hay duplicados en las columnas

In [73]:
df_games[df_games.duplicated(subset="app_name", keep=False)].sort_values(by="app_name").head(4)

,genres,app_name,release_date,price,developer,id_item
30545,"['Action', 'Free to Play', 'Indie', 'Strategy']",AirMech® Soundtrack,2012-11-13,9.99,Carbon Games,216173
18846,Sin Dato,AirMech® Soundtrack,Sin Dato,9.99,Sin Dato,599520
30177,['Action'],Aliens: Colonial Marines - Reconnaissance Pack,2013-05-07,29.99,Gearbox Software,219441
30176,['Action'],Aliens: Colonial Marines - Reconnaissance Pack,2013-05-07,Sin Dato,Gearbox Software,224850


In [74]:
df_games[df_games.duplicated(subset="app_name", keep=False)].sort_values(by="app_name").tail(4)

,genres,app_name,release_date,price,developer,id_item
27669,"['RPG', 'Strategy']",Warhammer Quest,2015-01-07,Sin Dato,"Twistplay,Rodeo Games",329270
27670,"['RPG', 'Strategy']",Warhammer Quest,2015-01-07,14.99,"Rodeo Games,Twistplay",326670
13892,['Action'],Wolfenstein II: The New Colossus,2017-10-26,59.99,Machine Games,612880
14571,['Action'],Wolfenstein II: The New Colossus,2017-10-26,59.99,Machine Games,612880


Aunque hay algunos registros que son claramente duplicados, otros que son más inciertos se dejaran para  
no perder información

In [75]:
eliminar_dup = [13892, 18846, 30176, 1067, 1508, 27669] # indices de los duplicados obvios

df_games = df_games.drop(eliminar_dup)

Como vemos en la siguiente celda, ya no se encuentran los mismos datos:

In [76]:
df_games[df_games.duplicated(subset="app_name", keep=False)].sort_values(by="app_name")

,genres,app_name,release_date,price,developer,id_item
31401,['Adventure'],Alter Ego,2010-08-03,9.99,bitComposer Games,63110
10911,"['Adventure', 'Casual', 'Indie', 'RPG', 'Simul...",Alter Ego,1986-05-01,7.99,Choose Multiple LLC,664780
31540,"['Action', 'Free to Play', 'Indie']",Altitude,2009-12-04,Free,Nimbly Games,41300
18194,Sin Dato,Altitude,2017-04-14,4.99,Sin Dato,620010
16453,Sin Dato,Archangel,Sin Dato,29.99,Sin Dato,553880
...,...,...,...,...,...,...
27087,"['Simulation', 'Sports']",Total Extreme Wrestling,2015-03-19,19.99,Grey Dog Software,344810
21898,['Simulation'],Ultimate Arena,2016-08-22,2.99,Triverske,385240
21697,"['Action', 'Indie']",Ultimate Arena,2016-09-05,11.15,"Kenneth Caselli,Gianmarco Rocco",436260
6580,"['Action', 'Indie']",WARZONE,2016-07-11,2.99,"NFA,Claw Games",494290


Vemos si hay duplicados en la columna id_item:

In [77]:
df_games[df_games.duplicated(subset="id_item", keep=False)]

,genres,app_name,release_date,price,developer,id_item


#### **Parte user_reviews**

Abrimos el archivo 'user_reviews_sentiment.csv'

In [16]:
df_reviews = pd.read_csv("./datasets/user_reviews_sentiment.csv", sep=";", encoding="utf-8")
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58458 entries, 0 to 58457
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             58458 non-null  object 
 1   item_id             58430 non-null  float64
 2   recommend           58430 non-null  object 
 3   sentimemt_analysis  58458 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.8+ MB


Observamos que la columna 'item_id' esta en el tipo de dato equivocado (float), ahora lo pasaremos a int  
Pero primero tendremos de encangarnos de los nulos (NaN) que tiene:

In [17]:
df_reviews[df_reviews["item_id"].isna()]

,user_id,item_id,recommend,sentimemt_analysis
137,gdxsd,NaN,NaN,1
177,76561198094224872,NaN,NaN,1
2558,76561198021575394,NaN,NaN,1
9956,cmuir37,NaN,NaN,1
13528,Jaysteeny,NaN,NaN,1
15212,ML8989,NaN,NaN,1
18792,76561198079215291,NaN,NaN,1
19784,76561198079342142,NaN,NaN,1
24521,76561198061996985,NaN,NaN,1
25715,76561198108286351,NaN,NaN,1


Debido a que estos registros carecen de información, los considero inútiles y procedo a eliminarlos:

In [18]:
df_reviews = df_reviews.dropna(subset=['item_id'])

In [19]:
df_reviews[df_reviews["item_id"].isna()]

,user_id,item_id,recommend,sentimemt_analysis


In [20]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58430 entries, 0 to 58457
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             58430 non-null  object 
 1   item_id             58430 non-null  float64
 2   recommend           58430 non-null  object 
 3   sentimemt_analysis  58430 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.2+ MB


Ahora si cambio el tipo de dato de item_id por int

In [21]:
df_reviews.insert(1, "id_item", df_reviews["item_id"].astype(int))
df_reviews.head(5)

,user_id,id_item,item_id,recommend,sentimemt_analysis
0,76561197970982479,1250,1250.0,True,2
1,76561197970982479,22200,22200.0,True,2
2,76561197970982479,43110,43110.0,True,2
3,js41637,251610,251610.0,True,2
4,js41637,227300,227300.0,True,2


Elimino la columna item_id, ya que sobra

In [22]:
df_reviews = df_reviews.drop("item_id", axis=1)
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58430 entries, 0 to 58457
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58430 non-null  object
 1   id_item             58430 non-null  int32 
 2   recommend           58430 non-null  object
 3   sentimemt_analysis  58430 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 2.0+ MB


#### **Parte user_items**

Abrimos el archivo 'user_reviews_sentiment.csv'

In [23]:
df_items = pd.read_csv("./datasets/user_items.csv", sep=";", encoding="utf-8")
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110796 entries, 0 to 5110795
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          int64  
 3   item_id           float64
 4   playtime_forever  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 195.0+ MB


### Buscamos nulos en cada columna

In [24]:
df_items[df_items["user_id"].isnull()]

,user_id,items_count,steam_id,item_id,playtime_forever


In [25]:
df_items[df_items["items_count"].isnull()]

,user_id,items_count,steam_id,item_id,playtime_forever


In [26]:
df_items[df_items["steam_id"].isnull()]

,user_id,items_count,steam_id,item_id,playtime_forever


In [27]:
df_items[df_items["item_id"].isnull()]

,user_id,items_count,steam_id,item_id,playtime_forever
3733,Wackky,0,76561198039117046,NaN,NaN
3849,76561198079601835,0,76561198079601835,NaN,NaN
6019,hellom8o,0,76561198117222320,NaN,NaN
6523,starkillershadow553,0,76561198059648579,NaN,NaN
7237,darkenkane,0,76561198058876001,NaN,NaN
...,...,...,...,...,...
5110251,76561198316380182,0,76561198316380182,NaN,NaN
5110252,76561198316970597,0,76561198316970597,NaN,NaN
5110253,76561198318100691,0,76561198318100691,NaN,NaN
5110787,XxLaughingJackClown77xX,0,76561198328759259,NaN,NaN


In [28]:
df_items[df_items["playtime_forever"].isnull()]

,user_id,items_count,steam_id,item_id,playtime_forever
3733,Wackky,0,76561198039117046,NaN,NaN
3849,76561198079601835,0,76561198079601835,NaN,NaN
6019,hellom8o,0,76561198117222320,NaN,NaN
6523,starkillershadow553,0,76561198059648579,NaN,NaN
7237,darkenkane,0,76561198058876001,NaN,NaN
...,...,...,...,...,...
5110251,76561198316380182,0,76561198316380182,NaN,NaN
5110252,76561198316970597,0,76561198316970597,NaN,NaN
5110253,76561198318100691,0,76561198318100691,NaN,NaN
5110787,XxLaughingJackClown77xX,0,76561198328759259,NaN,NaN


Como vemos hay usuarios que no poseen ningún item y por lo tanto no tienen horas jugadas  
Pero como no me gusta tener NaN's, voy a cambiarlos por un cero

In [29]:
df_items['item_id'] = df_items['item_id'].fillna(0)
df_items['playtime_forever'] = df_items['playtime_forever'].fillna(0)

In [30]:
df_items[df_items["item_id"].isnull()] # vemos el resultado

,user_id,items_count,steam_id,item_id,playtime_forever


In [31]:
df_items[df_items["playtime_forever"].isnull()] # vemos el resultado

,user_id,items_count,steam_id,item_id,playtime_forever


## **Gráficos**